# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data)
print(data['education'].unique())
print(data['children'].unique())
#display(data.head(30))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
[ 1  0  3  2 -1  4 20  5]


**Вывод**

В доходе стоят значения типа float, будет праблемотично разбивать данные на категории.
В days_employed обнаружены отрицательные значения, трудовой стаж не может быть минусовым.
Пропуски данных обноружены в колонках "days_employed" и "total_income".
Используются символы с разными регистрами, будет праблемотично разбивать данные на категории.
В колонке children обнаружены отрицательные значения и семьи с 20 детьми, детей не можеть меньше 0, а 20 детей - слишком подозрительно много.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
nan_data = data[data['total_income'].isna()]
display(nan_data)

display(data[data['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [3]:
print(nan_data['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [4]:
# Заполним пропущенные значения в колонке 'income_type' медианным значением в зависимости от типа занятости

employee_income_median = (data[data['income_type'] == 'сотрудник']['total_income'].median())
retiree_income_median = (data[data['income_type'] == 'пенсионер']['total_income'].median())
companion_income_median = (data[data['income_type'] == 'компаньон']['total_income'].median())
civil_servant_income_median = (data[data['income_type'] == 'госслужащий']['total_income'].median())
businessman_income_median = (data[data['income_type'] == 'предприниматель']['total_income'].median())
#unemployed_income_median = (data[data['income_type'] == 'безработный']['total_income'].median())
#decree_income_median = (data[data['income_type'] == 'в декрете']['total_income'].median())
#student_income_median = (data[data['income_type'] == 'студент']['total_income'].median())

print(employee_income_median)
print(retiree_income_median)


#data['total_income'] = data[data['income_type'] == 'сотрудник']['total_income'].fillna(employee_income_median)
#data['total_income'] = data[data['income_type'] == 'пенсионер']['total_income'].fillna(retiree_income_median)
#data['total_income'] = data[data['income_type'] == 'компаньон']['total_income'].fillna(companion_income_median)
#data['total_income'] = data[data['income_type'] == 'сотрудник']['total_income'].fillna(employee_income_median)
#data['total_income'] = data[data['income_type'] == 'сотрудник']['total_income'].fillna(employee_income_median)

data.update(data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 
                     'total_income'].fillna(employee_income_median))
data.update(data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 
                     'total_income'].fillna(retiree_income_median))
data.update(data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 
                     'total_income'].fillna(companion_income_median))
data.update(data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 
                     'total_income'].fillna(civil_servant_income_median))
data.update(data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 
                     'total_income'].fillna(businessman_income_median))


#data['days_employed'] = data['days_employed'].fillna(0)
#nan_data = data[data['total_income'].isna()]

#display(nan_data)
#print(nan_data['income_type'].value_counts())
#display(data.head(13))


data.info()

142594.39684740017
118514.48641164352
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
#заменим отрицательные числа на положительные в стаже
def my_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
data['days_employed'] = data['days_employed'].apply(my_pozitive)

In [6]:
#заменим отрицательные числа на положительные в колоке с детьми
data['children'] = data['children'].apply(my_pozitive)

In [7]:
# у некоторых заемщиков выявлено наличие 20 детей, что является маловероятным, тем более при возрасте некоторых родителей в 20 лет.
# земеним это количество детей на медианное, т.к. не понятно как эта ошибка появилась
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)


**Вывод**

Пропуски данных обноружены в колонках "days_employed" и "total_income", причём они находятся в одних и тех же строках.
Учитывая то, что мы не знаем должностей кредиторов, заполняем пропуски медианным уровнем дохода по каждому типу занятости.
Пропуски в трудовом стаже пропускаем или удаляем колонку целиком, т.к. данная информация, в текущем задании, нам не пригодится.

Предположительно, данную информацию заемщики целенаправленно не указывают. Необходимо обсудить с коллегами, в том числе с разработчиками, о возможных причинах и повышения мотивации в предоставлении достоверных данных потенциальными заемщиками.

### Замена типа данных

In [8]:
data['total_income'] = data['total_income'].astype('int')
#data['days_employed'] = data['days_employed'].astype('int')
data.info()
#display(data[data['days_employed'].isna()])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

В доходе стояли значения типа float - заменили их на int.

### Обработка дубликатов

In [9]:
#print(data['education'].unique())
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['purpose'] = data['purpose'].str.lower()

#display(data[data.duplicated()].sort_values(by='dob_years'))
data = data.drop_duplicates().reset_index(drop=True)
#print(data.duplicated().sum())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       19351 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Для того чтобы не пропустить дубликаты, привели все символы к нижнему регистру.
Проверили в Dataframe наличие дублирования строк. нам было необходимо выявить полное совпадение строк, поэтому был выбран вункция duplicates().
Возможными причинами дубликатов являются:
- ручное копирование
- обьединение нескольких баз данных
и маловероятное, но это все же разные люди или же один и тот же взявший кредит на одно и то же в течении года. Для исключения данной вероятности необходимы уникальные данные каждого челокека и дата с временем получения кредита.

Хотя остается вопрос, имеет ли свысл делать лемматизацию до выявления дубликатов?

### Лемматизация

In [10]:
#data["purpose"].iloc[0]

In [11]:
#''.join(m.lemmatize(data["purpose"].iloc[0])).strip()

In [12]:
#data["purpose"].apply(lambda x: ''.join(m.lemmatize(x)).strip())

In [13]:
from pymystem3 import Mystem
m = Mystem()


def my_lemmatization(text):
    text = ''.join(m.lemmatize(text)).strip()
    return text

In [14]:
data["purpose_lemm"] = data["purpose"].apply(my_lemmatization)

In [15]:
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем,операция с жилье
21450,0,343937.404131,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем,сделка с автомобиль
21451,1,2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,на покупка свой автомобиль


**Вывод**

Создали отдельную колонку с целями кредитования преведенными к единой лемме для возможности дальнейшей категоризации.

### Категоризация данных

In [16]:
#print(data['purpose_lemm'].value_counts())
def my_id_purpose(text):
    if 'автомобиль' in text:
        return 'автомобиль'
    if 'свадьба' in text:
        return 'свадьба'
    if 'коммерческий недвижимость' in text:
        return 'коммерческий недвижимость'
    if 'недвижимость' in text:
        return 'недвижимость/жилье'
    if 'жилье' in text:
        return 'недвижимость/жилье'
    if 'образование' in text:
        return 'образование'
    return 'прочее'  
        
data["purpose_category"] = data["purpose_lemm"].apply(my_id_purpose)
print(data['purpose_category'].value_counts())

#Создадим словарь целей кредитовая
#df_columns = ['my_purpose', "purpose_id"]
#df_data = [['автомобиль', 1], ['свадьба', 2], ['коммерческий недвижимость', 3], ['недвижимость/жилье', 4], ['образование', 5]]
#support_df = pd.DataFrame(data = df_data, columns = df_columns)
#display(support_df) 

недвижимость/жилье           9500
автомобиль                   4306
образование                  4013
свадьба                      2324
коммерческий недвижимость    1311
Name: purpose_category, dtype: int64


In [17]:
display(data.sort_values(by='total_income'))

# разобьем на категории исходя из уровня дохода на основании данных ЯндексДзена
def total_income_category(income):
    if income < 36000:
        return 'До 36000 руб, Бедные'
    elif 36000 <= income < 100000:
        return 'До 100000 руб, Выше бедности'
    elif 100000 <= income < 150000:
        return 'До 150000 руб, Нижний средний класс'
    elif 150000 <= income < 250000:
        return 'До 250000 руб, Предсредний класс'
    else:
        return 'От 250000 руб, Средний класс'
    
data["total_income_category"] = data["total_income"].apply(total_income_category)
print(data['total_income_category'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm,purpose_category
14555,0,359219.059341,57,среднее,1,женат / замужем,0,f,пенсионер,1,20667,недвижимость,недвижимость,недвижимость/жилье
12983,0,369708.589113,37,среднее,1,гражданский брак,1,m,пенсионер,0,21205,заняться высшим образованием,заниматься высокий образование,образование
16138,1,3642.820023,52,среднее,1,женат / замужем,0,m,сотрудник,0,21367,приобретение автомобиля,приобретение автомобиль,автомобиль
1598,0,359726.104207,68,среднее,1,гражданский брак,1,m,пенсионер,0,21695,на проведение свадьбы,на проведение свадьба,свадьба
14247,0,346602.453782,61,среднее,1,женат / замужем,0,f,пенсионер,0,21895,недвижимость,недвижимость,недвижимость/жилье
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17137,0,5734.127087,42,высшее,0,гражданский брак,1,m,компаньон,0,1711309,сыграть свадьбу,сыграть свадьба,свадьба
20742,0,4719.273476,61,среднее,1,не женат / не замужем,4,f,сотрудник,0,1715018,покупка жилья для семьи,покупка жилье для семья,недвижимость/жилье
9159,1,5248.554336,35,среднее,1,гражданский брак,1,m,сотрудник,0,1726276,дополнительное образование,дополнительный образование,образование
19548,1,2577.664662,39,высшее,0,женат / замужем,0,m,компаньон,1,2200852,строительство недвижимости,строительство недвижимость,недвижимость/жилье


До 150000 руб, Нижний средний класс    7160
До 250000 руб, Предсредний класс       7018
До 100000 руб, Выше бедности           4388
От 250000 руб, Средний класс           2813
До 36000 руб, Бедные                     75
Name: total_income_category, dtype: int64


**Вывод**

Данные категоризованы исходя из целей кредита, причем коммерческая недвижимости вымедена в отдельную категорию, т.к. ее берут ля осуществления предпренимательской деятельности, и уровня дохода заемщиков


## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
def my_category_children(text):
    if text < 1:
        return 'бездетные'
    return 'с детьми'
        
data["children_category"] = data["children"].apply(my_category_children)

dependence_on_children = data.pivot_table(index='children_category', values='debt', aggfunc=['count', 'sum'])
dependence_on_children['ratio'] = dependence_on_children['sum'] / dependence_on_children['count']
#dependence_on_children = pd.DataFrame()
#dependence_on_children['count_debt'] = data.groupby('children_category')['debt'].count()
#dependence_on_children['sum_debt'] = data.groupby('children_category')['debt'].sum()
#dependence_on_children['ratio'] = dependence_on_children['sum_debt'] / dependence_on_children['count_debt']

display(dependence_on_children)

,count,sum,ratio
,debt,debt,
children_category,,,
бездетные,14167,1071,0.075598
с детьми,7287,670,0.091945


**Вывод**


Зависимость есть, люди без детей являются более надежными заемщиками.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
dependence_on_family_status = pd.DataFrame()
dependence_on_family_status['count_debt'] = data.groupby('family_status')['debt'].count()
dependence_on_family_status['sum_debt'] = data.groupby('family_status')['debt'].sum()
dependence_on_family_status['ratio'] = dependence_on_family_status['sum_debt'] / dependence_on_family_status['count_debt']

display(dependence_on_family_status)

,count_debt,sum_debt,ratio
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
не женат / не замужем,2810,274,0.097509


**Вывод**

Зависимость есть. Люди которые в разводе или овдовели имеют меньше просрочек чем, те которые в данный момент замужем, а те кто не был женат или состаит в гражданском браке - являются самымы частыми неплатильщиками в данной выборке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
dependence_on_total_income_category = pd.DataFrame()
dependence_on_total_income_category['count_total_income_category'] = data.groupby('total_income_category')['debt'].count()
dependence_on_total_income_category['sum_total_income_category'] = data.groupby('total_income_category')['debt'].sum()
dependence_on_total_income_category['result_total_income_category'] = (dependence_on_total_income_category['sum_total_income_category'] / 
                                                             dependence_on_total_income_category['count_total_income_category'])

display(dependence_on_total_income_category)

,count_total_income_category,sum_total_income_category,result_total_income_category
total_income_category,,,
"До 100000 руб, Выше бедности",4388,350,0.079763
"До 150000 руб, Нижний средний класс",7160,624,0.087151
"До 250000 руб, Предсредний класс",7018,569,0.081077
"До 36000 руб, Бедные",75,4,0.053333
"От 250000 руб, Средний класс",2813,194,0.068966


**Вывод**

По бедным не хватает выборки для однозначного трактования данных, что означает что банк в данный момент редко кредитует данную категорию заемщиков.

По остальным категориям:
 * образцовыми платильщиками являются представители Среднего класса, с доходами от 250 т.р.
 * следом по надежности в порядке убывания, со значительным отрывом, представители с доходами "Выше бедности" и представители Предсреднего класса
 * худшими платильщиками являются представители Нижнего среднего класса, с доходами от 100 до 150 т.р. 
 


### Как разные цели кредита влияют на его возврат в срок?

In [21]:
dependence_on_purpose_category = pd.DataFrame()
dependence_on_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['debt'].count()
dependence_on_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['debt'].sum()
dependence_on_purpose_category['result_purpose_category'] = (dependence_on_purpose_category['sum_purpose_category'] / 
                                                             dependence_on_purpose_category['count_purpose_category'])

display(dependence_on_purpose_category)

,count_purpose_category,sum_purpose_category,result_purpose_category
purpose_category,,,
автомобиль,4306,403,0.093590
коммерческий недвижимость,1311,99,0.075515
недвижимость/жилье,9500,683,0.071895
образование,4013,370,0.092200
свадьба,2324,186,0.080034


**Вывод**

Чаще всего платят в срок с люди взявшие деньги на недвижимость вне зависимости на какую, хуже тех кто нужен кредит на авто и образование. Взявшие кредиты на свадьбу просрачивают реже чем те кто брал кредит на образование, но чаще чем люди взявшие деньги на недвижимость.

## Шаг 4. Общий вывод

Общий вывод - лучшим кредитором является представитель Среднего класса, с доходом от 250 т.р., в разводе или вдовец, без детей и берущий кредит на недвижимость. 